In [65]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
import plotly.express as px
import plotly.graph_objects as go

# To be available to render in GitHub
import plotly.io as pio
pio.renderers.default = "notebook_connected"

fig = ...

In [67]:
df = pd.read_csv("/kaggle/input/netflix-shows/netflix_titles.csv")

# Understanding the dataset

In [68]:
df.head()

In [69]:
df.info()

# Data Cleaning

Checking for missing values

In [70]:
df.isnull().sum()

The Missing Values are :
* director   : Not much use case at the moment, so dropping the whole column
* cast       : Too many different values, dropping the whole colums for now
* date_added : dropping the ones with no value
* Rating     : dropping it as it amounts to only 4 data left

**Dropping Director and Cast column**

In [71]:
df = df.drop(['director','cast'], axis = 1)

**Dropping null values on the date_added, rating, and duration**

In [72]:
df = df.dropna(subset=['date_added', 'rating', 'duration'])
df['country'] = df['country'].fillna(df['country'].mode()[0])

**Some entries contains more than one values, in this case separating them for easier access**

In [73]:
df['principal_country'] = df['country'].apply(lambda x: x.split(",")[0])
df['principal_country'].head()

**Fixing Data Types**

In [74]:
df.dtypes

**Create new entries based on year and month added from date_added column**

In [75]:
df["date_added"] = pd.to_datetime(df['date_added'])
df['year_added'] = df['date_added'].dt.year
df['month_added'] = df['date_added'].dt.month

In [76]:
#changing type into category form
df['type'] = pd.Categorical(df['type'])

# Data Visualization

In [77]:
df.head()

## Content Types on Netflix

In [78]:
fig = px.pie(df['type'].value_counts().reset_index(), values='type', names='index', color_discrete_sequence=px.colors.diverging.RdGy)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(title="<b>Movies & TV Shows distribution</b>")
fig.show()

* We can see that more than 2/3 of the content released on Netflix are movies at 69.7%.

## Growth of Netflix content over the years

In [79]:
added_year_df = df.loc[df['year_added'] > 2010].groupby(['year_added', 'type']).agg({'show_id': 'count'}).reset_index()


fig = go.Figure()
fig.add_trace(go.Scatter(
    x=added_year_df.loc[added_year_df['type'] == 'Movie']['year_added'],
    y=added_year_df.loc[added_year_df['type'] == 'Movie']['show_id'],
    mode='lines+markers',
    name='Movie: Year Added',
    marker_color= 'firebrick',
))
fig.add_trace(go.Scatter( 
    x=added_year_df.loc[added_year_df['type'] == 'TV Show']['year_added'], 
    y=added_year_df.loc[added_year_df['type'] == 'TV Show']['show_id'],
    mode='lines+markers',
    name='TV Show: Year Added',
    marker_color= 'DarkSlateGray',
))

fig.update_xaxes(categoryorder='total descending')
fig.update_layout(title="<b>Movies & TV Shows added over time</b>",template="plotly_white")
fig.show()

* It seems like movies are being added more over time in compared to tv shows. Starting at 2016, the numbers of movie being added had been increasing very rapidly and peaks at 2019

## Content distributed by countries

In [80]:
country_df = df['principal_country'].value_counts().reset_index().head(10)
dfh = df['principal_country'].value_counts().reset_index().head(3)

colors = ['#221f1f',]*10
colors[0] = colors[1] = colors[2] = 'Firebrick'
    

fig = px.bar(country_df, x='index', y='principal_country',
                   title='<b>Top 10 Countries of Netflix Content<b><br><sup>The top three have been highlighted</sup>',
                   labels={'index':'Countries', 'principal_country':'Number of Contents'},
                   template= "plotly_white",
                   )

fig.update_traces(marker_color=colors)
fig.update_traces(texttemplate='%{y}', textposition='outside')
fig.update_xaxes(categoryorder='total descending')
fig.show()

In [81]:
country_df2 = df['principal_country'].value_counts().reset_index()
country_df2 = country_df2[country_df2['principal_country'] /  country_df2['principal_country'].sum() > 0.005]

fig = px.pie(country_df2, values='principal_country', names='index', color_discrete_sequence=px.colors.diverging.RdGy,
             title='<b>Netflix content based of countries</b>',
             labels={'index':'Country', 'principal_country':'Count'})
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

* It's clear that United States has the most content on Netflix at almost 50% overall
* Suprisingly India comes as second on the number of content, beating United Kingdom
* We can also see that there is also a lot of content coming from Asian country

## Genre Analysis
**Defining the Genres**

In [82]:
df['genre'] = df['listed_in'].apply(lambda x :  x.replace(' ,',',').replace(', ',',').split(','))

**Divides the genres into two types of content**

In [83]:
movie_df = df[df['type'] == 'Movie']
show_df = df[df['type'] == 'TV Show']

In [84]:
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.colors

def calculate_mlb(series):
    mlb = MultiLabelBinarizer()
    mlb_df = pd.DataFrame(mlb.fit_transform(series), columns=mlb.classes_, index=series.index)
    return mlb_df

In [85]:
def top_genres(df, title):
    colors = ['#221f1f',]*20
    colors[0] = colors[1] = colors[2] = colors[3] = colors[4] = 'Firebrick'
    
    genres_df = calculate_mlb(df['genre'])
    tdata = genres_df.sum().sort_values(ascending=False)
    
    Types = []
    for i in df['genre']: Types += i
    Types = set(Types)
    print("There are {} types in the Netflix {} Dataset".format(len(Types),title))    
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=tdata.index,
        y=tdata.values,
        marker=dict(color="Firebrick")
    ))
    fig.update_traces(marker_color=colors)
    fig.update_xaxes(categoryorder='total descending')
    fig.update_traces(texttemplate='%{y}', textposition='outside')
    fig.update_layout(title="<b>Top {} Genres</b><br><sup>The top five have been highlighted</sup>".format(title), template="plotly_white")
    fig.show()

In [86]:
def genre_heatmap(df, title):
    cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ['#221f1f', 'Firebrick','LavenderBlush'])
    
    genres_df = calculate_mlb(df['genre'])
    movie_corr = genres_df.corr()
    movie_mask = np.zeros_like(movie_corr, dtype=np.bool)
    movie_mask[np.triu_indices_from(movie_mask)] = True
    fig, ax = plt.subplots(figsize=(10, 7))
    pl = sns.heatmap(movie_corr, mask=movie_mask, cmap= cmap, vmax=.3, vmin=-.3, center=0, linewidths=.25,
                 cbar_kws={"shrink": 0.6})
    ax.set_title("Genre Correlation in {}".format(title), fontweight='bold',fontsize=15)
    plt.show()

**Let's Start the analysis on the Movies genre**

In [87]:
top_genres(movie_df, 'Movies')

In [88]:
genre_heatmap(movie_df, 'Movies')

From both analyses, we can see that :
* The top 5 genres for movies are International Movies, Dramas, Comedies, Documentaries and Action & Adventure.
* It seems that most International Movies and Indeoendent Movies tend to be Dramas.
* Interestingly, International Movies are rarely the in the Childen's genre.

**Now on to the TV Shows**

In [89]:
top_genres(show_df, 'TV Shows')

In [90]:
genre_heatmap(show_df, 'TV Shows')

From both analyses, we can see that :
* The top 5 genres for TV Shows are International TV Shows, TV Dramas, TV Comedies, Crime TV Shows and Kids' TV.
* It seems that most of the Sci-fi & Fantasy shows are about Action & Adventure.
* Different from Movies, most International TV Shows are Romance genre.
* In TV Shows, Mysteries and Horrors are the most common along with Docuseries with Science & Nature Genre.
* Seems like the Kids' genre doesn't goes well with Dramas and International TV Shows.

## Duration analysis
### Movies Length Analysis

In [91]:
movie_duration = movie_df['duration'].apply(lambda x: int(x.split(' ')[0])).astype(int).reset_index()
movie_duration["Count"] = movie_duration['duration']**0

fig = px.histogram(movie_duration, x="duration", y="Count",
                    title='<b>Movies Duration Distribution<b>',
                    labels={'duration':'<b>Minutes<b>'},
                    template= "plotly_white",
                    color_discrete_sequence=['firebrick']
)
fig.update_layout(yaxis_title=None)
fig.show()

#Finding the mean, min, and max duration
movie_mean = movie_duration['duration'].mean()
movie_low = movie_duration['duration'].min()
movie_high = movie_duration['duration'].max()

print("The average duration of movies releases in Netflix is {} minutes".format(round(movie_mean)))
print("The shortest movie released in Netflix is {} minutes".format(movie_low))
print("The longest movie released in Netflix is {} minutes".format(movie_high))

### TV Shows Series Analysis

In [92]:
#show_df1 tallies TV Shows based on duration
show_df1 = show_df.groupby('duration').size().reset_index(name='Count')

fig = px.bar(show_df1, x='duration', y= 'Count',
                   title='<b>TV Shows Series Distribution<b>',
                   labels={'duration':'Duration of the Show'},
                   template= "plotly_white",
                   )

fig.update_traces(marker_color='firebrick')
fig.update_traces(texttemplate='%{y}', textposition='outside')
fig.update_layout(xaxis={'categoryorder': 'total descending'})
fig.show()

longest_show = show_df[show_df['duration'].str.contains('17 Seasons')]['title'].iloc[-1]
print('The longest TV Shows on Netflix is {} at 17 seasons'.format(longest_show, index=False))

# Netflix Title Overview with Word Cloud

In [93]:
from wordcloud import WordCloud
from PIL import Image
import random

# Custom colour map based on Netflix palette
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ['#221f1f', 'Firebrick'])

text = str(list(df['title'])).replace(',', '').replace('[', '').replace("'", '').replace(']', '').replace('.', '')

mask = np.array(Image.open('../input/images2/netflix-n.jpg'))

wordcloud = WordCloud(background_color = 'white', width = 1000,  height = 1000, max_words = 150, colormap=cmap, mask = mask).generate(text)

plt.figure(figsize=(12, 8))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.show()

## Thanks for reading